In [1]:
import torch
from transformers import GPT2Tokenizer
from trl_custom import PPOConfig, create_reference_model, AutoModelForCausalLMWithValueHead, PPOTrainer
from trl_custom.core import respond_to_batch

# 1. load a pretrained model
model_encoder = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
#model_decoder = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')

model_ref = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# 2. initialize trainer
ppo_config = {'batch_size': 16}
config = PPOConfig(**ppo_config)
ppo_encoder_trainer = PPOTrainer(config, model_encoder, model_ref, tokenizer)
#ppo_decoder_trainer = PPOTrainer(config, model_decoder, model_ref, tokenizer) # remove model_ref?

/home/gridsan/bwright/PycharmProjects/StegLLM/trl_custom/trainer/ppo_trainer.py:223: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [2]:
import torch
from transformers import GPT2Tokenizer

class StegEnv():
    def __init__(self):
        self.prompts = [" 0 1 0 1", "This morning I went to the ", "The weather today is ", "What is your favorite "]
        self.secrets = [' 1 1 0 1',
                        ' 1 0 1 0',
                        ' 0 1 0 1',
                        ' 1 1 1 1',
                        ' 0 0 1 1',
                        ' 1 1 1 0',
                        ' 1 0 0 1',
                        ' 0 0 0 1',
                        ' 0 1 0 0',
                        ' 1 0 1 1',
                        ' 0 1 1 1',
                        ' 0 0 0 0',
                        ' 1 1 0 0',
                        ' 0 1 1 0',
                        ' 0 0 1 0',
                        ' 1 0 0 0']

        self.prompt_batch = None
        self.secret_batch = None

        self.batch_size = 16 # must be the same as value set in ppo_config

        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def _get_queries(self):
        return ["secret:" + secret + " prompt:" + prompt for secret, prompt in zip(self.secret_batch, self.prompt_batch)]
    
    def _get_obs(self):
        queries = self._get_queries()
        query_pt = [tokenizer.encode(query, return_tensors="pt").squeeze() for query in queries]
        prompt_pt = [tokenizer.encode(prompt, return_tensors="pt").squeeze() for prompt in self.prompt_batch]
        return {
            "query": queries,
            "query_pt": query_pt,
            "prompt": self.prompt_batch,
            "prompt_pt": prompt_pt,
            "secret": self.secret_batch
        }

    def reset(self, ):
        prompt_idxs = torch.randint(len(self.prompts), size=(self.batch_size,))
        secret_idxs = torch.randint(len(self.secrets), size=(self.batch_size,))

        self.prompt_batch = [self.prompts[idx] for idx in prompt_idxs]
        self.secret_batch = [self.secrets[idx] for idx in secret_idxs]

        print(self.prompt_batch, self.secret_batch)

        obs = self._get_obs()

        return obs

    def _reward_function(self, responses_encoder, responses_decoder):

        rewards_encoder = []
        rewards_decoder = []

        for response_enc, response_dec, secret in zip(responses_encoder, responses_decoder, self.secret_batch):
            secret_pt = tokenizer.encode(secret, return_tensors="pt").squeeze()

            # if response_dec_txt == secret:
            #     reward_enc, reward_dec = 1.0, 1.0
            # else:
            #     reward_enc, reward_dec = 0.0, 0.0

            reward_enc = reward_dec = (secret_pt == response_dec).sum()

            rewards_encoder.append(torch.tensor(reward_enc))
            rewards_decoder.append(torch.tensor(reward_dec))

        return (rewards_encoder, rewards_decoder)

    def step(self, responses_encoder, responses_decoder):

        reward = self._reward_function(responses_encoder, responses_decoder)
        return reward



In [3]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

def get_responses(obs, ppo_encoder_trainer):

    enc_response_len = 4
    dec_response_len = 4
    
    encoder_queries = obs['query_pt']
    decoder_queries = []
    encoder_responses = []
    decoder_responses = []

    for query_encoder_pt, prompt_encoder_pt in zip(encoder_queries, obs['prompt_pt']):

        # get encoder response
        generation_kwargs["max_new_tokens"] = enc_response_len
        response_encoder_pt = ppo_encoder_trainer.generate(query_encoder_pt, **generation_kwargs)
        response_encoder_pt = response_encoder_pt.squeeze()[-enc_response_len:]
        encoder_responses.append(response_encoder_pt)

        # construct decoder query
        query_decoder_pt = torch.cat((prompt_encoder_pt, response_encoder_pt))
        decoder_queries.append(query_decoder_pt)

        # get decoder response
        generation_kwargs["max_new_tokens"] = dec_response_len
        response_decoder_pt = ppo_encoder_trainer.generate(query_decoder_pt, **generation_kwargs)
        response_decoder_pt = response_decoder_pt.squeeze()[-dec_response_len:]
        
        # response_decoder_txt = tokenizer.decode(response_decoder_pt.squeeze())
        decoder_responses.append(response_decoder_pt)

    return encoder_queries, encoder_responses, decoder_queries, decoder_responses

def train(ppo_encoder_trainer):

    num_episodes = 10
    env = StegEnv()

    for episode in range(num_episodes):
        obs = env.reset()
        encoder_queries, encoder_responses, decoder_queries, decoder_responses = get_responses(obs, ppo_encoder_trainer)
        encoder_rewards, decoder_rewards = env.step(encoder_responses, decoder_responses)

        print([tokenizer.decode(res) for res in encoder_responses])
        print([tokenizer.decode(res) for res in decoder_responses])
        print(encoder_rewards, decoder_rewards)

        stats1 = ppo_encoder_trainer.step(encoder_queries, encoder_responses, encoder_rewards,
                                          decoder_queries, decoder_responses, decoder_rewards)
        #stats2 = ppo_decoder_trainer.step(decoder_queries, decoder_responses, decoder_rewards)
        # log the stats?
    

In [4]:
train(ppo_encoder_trainer)

[' 0 1 0 1', 'What is your favorite ', 'This morning I went to the ', ' 0 1 0 1', 'What is your favorite ', 'What is your favorite ', 'What is your favorite ', 'What is your favorite ', 'This morning I went to the ', 'What is your favorite ', 'This morning I went to the ', 'The weather today is ', ' 0 1 0 1', 'What is your favorite ', 'The weather today is ', ' 0 1 0 1'] [' 0 1 0 0', ' 1 1 0 0', ' 0 1 1 1', ' 1 0 1 0', ' 1 0 0 1', ' 0 0 0 1', ' 0 0 0 1', ' 0 0 1 0', ' 0 1 0 0', ' 1 0 1 0', ' 1 1 1 0', ' 1 0 1 1', ' 0 1 1 0', ' 0 1 0 0', ' 1 0 1 1', ' 0 0 1 1']


/state/partition1/slurm_tmp/22505819.0.0/ipykernel_29273/2312622544.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards_encoder.append(torch.tensor(reward_enc))
/state/partition1/slurm_tmp/22505819.0.0/ipykernel_29273/2312622544.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards_decoder.append(torch.tensor(reward_dec))


[' user: 24 4', 'irl? metadata Scan', 'ivanATION@&', ' prompt on 0 1', '!!! Nasty stuff', 'irc to speak of', 'ime? It may', '\xa0Rock of Ages', 'vernacular David refused', '?????Natsume', 'アウニャ', '\xa0cool and sunny', ' case... case item', "???? I'm Legion", '_______. There are', ' prompt: 1 0']
[' 46.66%', 'ner: Menu menu', 'WWU branch of', ' 0 0 0 1', ' both trans folks agree', '?\n\nGuest', ' be one he or', '? Have you got', ' to publish and I', ': nGunno', 'ンタル�', '. Right now I', ' 1: case case', ' _ fan Aeris', ' a lot of snow', ' 0 1 0 1']
[tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(2)] [tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(2)]


RuntimeError: mean(): input dtype should be either floating point or complex dtypes. Got Long instead.

In [ ]:
results1